In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.0.1"
#r "nuget: Microsoft.SemanticKernel.Connectors.Chroma, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Qdrant, 1.0.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha"
#r "nuget: System.Linq.Async, 6.0.1"

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Connectors.Chroma, 1.0.1-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.0.1-alpha System.Linq.Async, 6.0.1

In [6]:
#!import ../Settings/AppConfig.cs

AppConfig.aoaiSettings apiConfig = AppConfig.GetSecrets();

// Get AOAI values
string mydeploymentName = apiConfig.deployTurbo;
string mydeployEmbed = apiConfig.deployEmbed;
string myendpoint = apiConfig.endpoint;
string myAPIKey = apiConfig.APIKey;

//NOTE: If you run the below writeline command, be sure to clear the output before pushing to repo!!
//Console.WriteLine(mydeploymentName + " & " + myendpoint + " & " + myAPIKey);

In [3]:
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Chroma;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion(mydeploymentName, myendpoint, myAPIKey);


var kernel = builder.Build();

In [9]:
#pragma warning disable SKEXP0003, SKEXP0011, SKEXP0022, SKEXP0052

using Microsoft.SemanticKernel.Connectors.OpenAI;

var memoryBuilder = new MemoryBuilder();
memoryBuilder.WithAzureOpenAITextEmbeddingGeneration(mydeployEmbed, myendpoint, myAPIKey);

var chromaMemoryStore = new ChromaMemoryStore("http://127.0.0.1:8000");
memoryBuilder.WithMemoryStore(chromaMemoryStore);

var memory = memoryBuilder.Build();

In [13]:
const string MemoryCollectionName = "aboutMe";

await memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Fred Flintstone");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I am married and my wifes name is Wilma");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I currently live in the town of Bedrock");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "My best friend Barney Rubble lives next to me with his wife Betty.");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info5", text: "I currently work as a crane operator at Slate Rock and Gravel Company");

In [15]:
var questions = new[]
{
    "what is my name?",
    "where do I live?",
    "who is my neighbor?",
    "Am I married?",
    "what do I do for work?",
};

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q, limit: 1, minRelevanceScore: 0.5).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text);
}

what is my name? My name is Fred Flintstone
where do I live? I currently live in the town of Bedrock
who is my neighbor? My best friend Barney Rubble lives next to me with his wife Betty.
Am I married? I am married and my wifes name is Wilma
what do I do for work? I currently work as a crane operator at Slate Rock and Gravel Company
